## Hardware check

In [1]:
# gpu check
! nvidia-smi

Thu May 30 11:23:52 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-32GB           On  |   00000000:3A:00.0 Off |                    0 |
| N/A   30C    P0             41W /  300W |      17MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# check number of cores
import multiprocessing

cores = multiprocessing.cpu_count() 
cores

80

## Environment Prep

In [4]:
cd /pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis/

/pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis


In [5]:
pwd

'/pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis'

In [ ]:
import pickle
from simpletransformers.language_representation import RepresentationModel
from simpletransformers.config.model_args import ModelArgs
import resources.smart_cond as sc

In [1]:
import pickle

### Version Check

In [6]:
! python --version

Python 3.9.7


In [7]:
import tensorflow as tf
print(tf. __version__)

2.12.0


In [8]:
pickle.format_version

'4.0'

## Load Data

In [5]:
# ! unzip Data/sepsis_removed_1.pkl.zip

Archive:  Data/sepsis_removed_1.pkl.zip
  inflating: sepsis_removed_1.pkl    


In [7]:
data_path = 'Data/sepsis_removed_0.pkl'
pkl = pickle.load(open(data_path, 'rb'))
data = pkl[0]

In [13]:
from tqdm import tqdm

In [38]:
len(data)

82447326

In [9]:
data

,ts_ind,hour,variable,value,TABLE,mean,std
0,10223,467.816667,Text,Admission Date: [**2119-5-4**] D...,noteevents,1.000000,1.000000
1,18407,28.016667,Text,Admission Date: [**2112-12-8**] ...,noteevents,1.000000,1.000000
2,40300,155.166667,Text,Admission Date: [**2194-7-18**] ...,noteevents,1.000000,1.000000
3,23747,52.383333,Text,Admission Date: [**2194-1-7**] D...,noteevents,1.000000,1.000000
4,2357,73.133333,Text,Admission Date: [**2186-6-7**] Discharge ...,noteevents,1.000000,1.000000
...,...,...,...,...,...,...,...
82886223,57281,20.400000,MBP,0.195381,chart,78.552377,17.645628
82886224,57281,20.400000,O2 Saturation,-0.678068,chart,96.820961,4.160290
82886225,57281,20.400000,RR,0.179866,chart,26.278501,15.130729
82886226,57281,20.400000,SBP,-0.404061,chart,120.239648,25.341836


In [47]:
# i = -1
# for text in data['value'][:100000]:
#     i += 1
#     if 'sepsis' in text:
#         print(i)
#         print('-')

In [10]:
'sepsis' in data['value'][3]

True

In [6]:
data_path = 'Data/sepsis_removed_1.pkl'
pkl = pickle.load(open(data_path, 'rb'))
data = pkl[0]
del pkl

# text_data = data[data['variable'] == 'Text']
# text_data

FileNotFoundError: [Errno 2] No such file or directory: 'Data/sepsis_removed_1.pkl'

In [ ]:
texts = text_data['value']
texts

0          Admission Date:  [**2119-5-4**]              D...
1          Admission Date:  [**2112-12-8**]              ...
2          Admission Date:  [**2194-7-18**]              ...
3          Admission Date:  [**2194-1-7**]              D...
4          Admission Date:  [**2186-6-7**]     Discharge ...
                                 ...                        
1407425    NPN\n\n\n#1  Infant remains in RA with O2 sats...
1407426    Neonatology\nDOL #5, CGA 36 weeks. \n\nCVR: Co...
1407427    Family Meeting Note\nFamily meeting held with ...
1407428    NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...
1407429    NPN 1800\nNursing Addendum:\n[**Known lastname...
Name: value, Length: 1407430, dtype: object

In [7]:
batch = texts[700000:].tolist()
len(batch)

707430

In [8]:
model_args = ModelArgs(max_seq_length=512, silent = True)
model = RepresentationModel(
    "bert", "emilyalsentzer/Bio_ClinicalBERT", args=model_args)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTextRepresentation: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
# time the process
import time
tic = time.perf_counter()

In [ ]:
# features = model.encode_sentences(batch, combine_strategy="mean")
# features = model.encode_sentences(batch, combine_strategy=None)
# CLS
features = model.encode_sentences(batch, combine_strategy=0)

In [ ]:
# batch 1, 4 gpus
toc = time.perf_counter()
print(f"Time consumed generating embeddings for {toc - tic:0.4f} seconds")

In [ ]:
features.shape

In [ ]:
pickle.dump([features], open('features_CLS_707430.pkl','wb'))

### update data table with an `emb` column

In [3]:
import pickle
import numpy as np

In [4]:
# read embeddings
batch_0 = pickle.load(open('features_CLS_700000.pkl', 'rb'))
batch_1 = pickle.load(open('features_CLS_707430.pkl', 'rb'))

emb_0 = batch_0[0]
emb_1 = batch_1[0]

In [5]:
emb = np.concatenate([emb_0, emb_1])

In [6]:
emb.shape

(1407430, 768)

In [9]:
pkl = [emb]

In [10]:
pickle.dump(pkl, open('Data/CLS_emb_keywords_reserved.pkl','wb'))

In [11]:
data_path = 'Data/CLS_emb_keywords_reserved.pkl'
pkl = pickle.load(open(data_path, 'rb'))
data = pkl[0]

In [12]:
pkl[-1].shape

(1407430, 768)